Multi Processing in python

In [33]:
from multiprocessing import Process, Queue, Value

def func(numbers, q):
    for i in numbers:
        q.put(i * i)

if __name__ == "__main__":
    num = [2, 4, 6]
    q = Queue()
    
    p = Process(target=func, args=(num, q))
    p.start()
    p.join()

    while not q.empty():
        print(q.get())


4
16
36


Multi Processing using Pipe
2 way communication

In [1]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


LOCKS for critical sections

In [19]:
from multiprocessing import Process, Pipe, Value,Lock
import time


def discount(conn,balance):

    
    disbalance = balance.value+1000
    conn.send(disbalance)
    conn.close()

def withdraw(balance,lock):
    time.sleep(0.001)
    lock.acquire()
    for i in range(100):
        balance.value-=1
        
    lock.release()

def deposit(balacne,lock):

    lock.acquire()
    time.sleep(0.001)
    for i in range(50):
        balance.value+=1

    lock.release()



if __name__ == '__main__':
    
    balance = Value('i',300)
    lock = Lock()
    p1 = Process(target = deposit, args = (balance,lock))
    p2 = Process(target = withdraw, args = (balance,lock))

    p1.start()
    p2.start()
    #p2.start() 

    p1.join()
    p2.join()
    
    print("balance rem is : ", balance.value)


    P_conn, C_conn = Pipe()
    p3 = Process(target = discount,args = (C_conn,balance))
    p3.start()
    print("after discount balance is : ",P_conn.recv())

    p3.join()




balance rem is :  250


after discount balance is :  1250


async

In [32]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import time

async def fetch(session, url):
    async with session.get(url) as response:
        content = await response.text()
        return url, content

async def extract_headings(content):
    soup = BeautifulSoup(content, 'html.parser')
    headings = {
        'h1': [h1.get_text() for h1 in soup.find_all('h1')],
        'h2': [h2.get_text() for h2 in soup.find_all('h2')],
        'h3': [h3.get_text() for h3 in soup.find_all('h3')],
        'h4': [h4.get_text() for h4 in soup.find_all('h4')],
        'h5': [h5.get_text() for h5 in soup.find_all('h5')],
        'h6': [h6.get_text() for h6 in soup.find_all('h6')]
    }
    return headings

async def main():
    urls = [
        'https://www.youtube.com',
        'https://www.python.org',
        'https://www.github.com',
        'https://www.wikipedia.org'
    ]

    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        results = await asyncio.gather(*tasks)

        extract_tasks = [extract_headings(content) for url, content in results]
        extracted_headings = await asyncio.gather(*extract_tasks)

    for (url, content), headings in zip(results, extracted_headings):
        print(f"Headings from {url}:")
        for tag, texts in headings.items():
            if texts:
                print(f"{tag.upper()}:")
                for text in texts:
                    print(f"  - {text}")
        print()


loop = asyncio.get_event_loop()
if loop.is_running():
    task = loop.create_task(main())
    await task
else:
    asyncio.run(main())


Headings from https://www.example.com:
H1:
  - Example Domain

Headings from https://www.python.org:
H1:
  - 


  - Functions Defined
  - Compound Data Types
  - Intuitive Interpretation
  - All the Flow You’d Expect
  - Quick & Easy to Learn
H2:
  - Get Started
  - Download
  - Docs
  - Jobs
  - Latest News
  - Upcoming Events
  - Success Stories
  - Use Python for…
  - 
>>> Python Enhancement Proposals (PEPs): The future of Python is discussed here.
 RSS

  - 
>>> Python Software Foundation


Headings from https://www.github.com:
H1:
  - Search code, repositories, users, issues, pull requests...
  - 
        Provide feedback
      
  - 
        Saved searches
      
  - 
Let’s build from here

H2:
  - Navigation Menu
  - Use saved searches to filter your results more quickly
  - Productivity
  - GitHub Copilot empowers developers to complete tasks 55% faster with contextualized AI coding assistance across workflows.
  - Application security
  - GitHub Advanced Security enables you to

/home/de/.local/lib/python3.10/site-packages/bs4/element.py:955: RuntimeWarning: coroutine 'main' was never awaited
  u.setup()


Dunders method,

Dunder methods, short for "double underscore" methods, are special methods in Python that begin and end with double underscores (e.g., __init__, __str__). They are used to define how objects of a class behave with built-in Python operations and functions. These methods allow you to customize the behavior of your objects in various contexts.

In [34]:
class MyClass:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f"MyClass({self.value})"

    def __str__(self):
        return str(self.value)

    def __add__(self, other):
        return MyClass(self.value + other.value)

    def __eq__(self, other):
        return self.value == other.value

    def __len__(self):
        return len(str(self.value))

    def __call__(self, new_value):
        self.value = new_value

obj1 = MyClass(10)
obj2 = MyClass(20)

print(repr(obj1))
print(str(obj1))

result = obj1 + obj2
print(repr(result))

print(obj1 == obj2)
print(len(obj1))

obj1(30)
print(obj1)


MyClass(10)
10
MyClass(30)
False
2
30


Garbage collection 

Garbage collection in Python is the process of automatically reclaiming memory by deleting objects that are no longer in use. Python’s garbage collector primarily uses reference counting and a cyclic garbage collector to manage memory.

In [36]:
import gc

class MyClass:
    def __init__(self, name):
        self.name = name
        print(f"Object {self.name} created")

    def __del__(self):
        print(f"Object {self.name} deleted")

def create_objects():
    obj1 = MyClass("A")
    obj2 = MyClass("B")
    obj3 = MyClass("C")
    
    obj1.ref = obj2
    obj2.ref = obj3
    obj3.ref = obj1

    del obj1
    del obj2
    del obj3

create_objects()
gc.collect()  



Object A created
Object B created
Object C created
Object A deleted
Object B deleted
Object C deleted


13